In [55]:
import pandas as pd
import numpy as np

In [56]:
from bs4 import BeautifulSoup
import requests

In [57]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup=BeautifulSoup(source,'html.parser')



In [58]:
table=soup.find('table')

In [59]:
soup.tr.text

'\nPostal code\n\nBorough\n\nNeighborhood\n'

In [60]:
columns=['Postalcode','Borough','Neighborhood']
df=pd.DataFrame(columns=columns)

In [61]:
df

,Postalcode,Borough,Neighborhood


### Appending the data into the dataframe by going through each table row and then append  the table data into the row 

In [62]:
for tr in table.find_all('tr'):
    row=[]
    for td in tr.find_all('td'):
        row.append(td.text.strip())
    if len(row)==3:
        df.loc[len(df)]=row

In [63]:
df.head()

,Postalcode,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [64]:
indexNames = df[ df['Borough'] =='Not assigned'].index
df.drop(indexNames,inplace=True)

In [66]:
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [69]:
df[df['Neighborhood']=='Not assigned']

,Postalcode,Borough,Neighborhood


In [76]:
df.loc[df['Neighborhood']=='Not assigned','Neighborhood']=df['Borough']
df.head(10)

,Postalcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
8,M9A,Etobicoke,Islington Avenue
9,M1B,Scarborough,Malvern / Rouge
11,M3B,North York,Don Mills
12,M4B,East York,Parkview Hill / Woodbine Gardens
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [73]:
result = df.groupby(['Postalcode','Borough'], sort=False).agg( ', '.join)
df_new=result.reset_index()
df_new.head(10)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park / Harbourfront
3,M6A,North York,Lawrence Manor / Lawrence Heights
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,Malvern / Rouge
7,M3B,North York,Don Mills
8,M4B,East York,Parkview Hill / Woodbine Gardens
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [77]:
df_new.shape

(103, 3)

In [78]:
df_coord=pd.read_csv('Geospatial_Coordinates.csv')
df_coord.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [80]:
df_coord.rename(columns={'Postal Code':'Postalcode'},inplace=True)

In [82]:
df_coord.head()

,Postalcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [84]:
toronto_df=pd.merge(df_new,df_coord[['Postalcode','Latitude','Longitude']],on='Postalcode')

In [85]:
toronto_df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494


In [86]:
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
print('Libraries Imported')

Libraries Imported


In [87]:
address='Toronto, ON'
geolocator=Nominatim(user_agent='Toronto')
location=geolocator.geocode(address)
latitude_tor=location.latitude
longitude_tor=location.longitude
print('latitude {},longitude {}'.format(latitude_tor,longitude_tor))

latitude 43.6534817,longitude -79.3839347


In [89]:
map_tor=folium.Map(location=[latitude_tor,longitude_tor],zoom_start=10)

for lat,long,borough,neighborhood in zip(toronto_df['Latitude'],toronto_df['Longitude'],toronto_df['Borough'],toronto_df['Neighborhood']):
    label='{},{}'.format(neighborhood,borough)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=label,
    color='Blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_tor)
map_tor

In [92]:
CLIENT_ID='PRFI05SBWGTTQT2WCCN5FMBAIG0Y2INGQBFPPMDEU3J0GZDA'
CLIENT_SECRET='H0IO3LMRZKIGO20CV1WNJ1NXJ3H0SYSREGFDDJYWU12OZVBP'
VERSION='20180605'
radius=500
LIMIT=100


In [93]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT        
        )

In [94]:
result=requests.get(url).json()
result

{'meta': {'code': 200, 'requestId': '5e8c0b2a963d29001b80bf81'},
 'response': {'headerLocation': 'Islington - City Centre West',
  'headerFullLocation': 'Islington - City Centre West, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 13,
  'suggestedBounds': {'ne': {'lat': 43.6333408045, 'lng': -79.51479402615583},
   'sw': {'lat': 43.6243407955, 'lng': -79.52720477384419}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc9f9b6b6c49c7469688f91',
       'name': 'South St. Burger',
       'location': {'address': '1020 Islington Ave',
        'lat': 43.63131374571208,
        'lng': -79.51840794740386,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.63131374571208,
          'lng': -79.51840794740386}],
        'distance': 345

In [127]:
def getnearbyvenue(names,latitudes,longitudes,radius=500):
    venues_list=[]
    for name,lat,lng in zip(names,latitudes,longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius,
        LIMIT        
        )
        results=requests.get(url).json()['response']['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
    
        
        

In [129]:
toronto_venues=getnearbyvenue(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park / Harbourfront
Lawrence Manor / Lawrence Heights
Queen's Park / Ontario Provincial Government
Islington Avenue
Malvern / Rouge
Don Mills
Parkview Hill / Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park / Princess Gardens / Martin Grove / Islington / Cloverdale
Rouge Hill / Port Union / Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate / Bloordale Gardens / Old Burnhamthorpe / Markland Wood
Guildwood / Morningside / West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor / Wilson Heights / Downsview North
Thorncliffe Park
Richmond / Adelaide / King
Dufferin / Dovercourt Village
Scarborough Village
Fairview / Henry Farm / Oriole
Northwood Park / York University
East Toronto
Harbourfront East / Union Station / Toronto Islands
Little Portugal / Trinity
Kennedy Park / Ionview / East Birchmount Park
Bayview Village
Do

Lets check the data and shape of the Dataframe

In [136]:
toronto_venues.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Pho Com Viet Nam,43.756631,-79.518336,Vietnamese Restaurant
1,Parkwoods,43.753259,-79.329656,Pizza Hut,43.756340,-79.517818,Pizza Place
2,Parkwoods,43.753259,-79.329656,KFC,43.756600,-79.518100,Fast Food Restaurant
3,Parkwoods,43.753259,-79.329656,Subway,43.756171,-79.518251,Sandwich Place
4,Parkwoods,43.753259,-79.329656,The Beer Store,43.756094,-79.516239,Beer Store
5,Parkwoods,43.753259,-79.329656,Tim Hortons,43.754344,-79.527024,Coffee Shop
6,Parkwoods,43.753259,-79.329656,Tim Hortons,43.756128,-79.516266,Coffee Shop
7,Parkwoods,43.753259,-79.329656,Planet Fitness,43.757538,-79.519610,Gym / Fitness Center
8,Parkwoods,43.753259,-79.329656,Hwy 400 at Finch W.,43.754399,-79.526967,Road
9,Parkwoods,43.753259,-79.329656,Jian Hing Supermarket,43.756673,-79.518444,Grocery Store


In [137]:
toronto_venues.shape

(710, 7)

Lets check how many values were returned for each neighborhood

In [135]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
Alderwood / Long Branch,4,4,4,4,4,4
Bathurst Manor / Wilson Heights / Downsview North,19,19,19,19,19,19
Bayview Village,3,3,3,3,3,3
Bedford Park / Lawrence Manor East,1,1,1,1,1,1
Berczy Park,20,20,20,20,20,20
Birch Cliff / Cliffside West,3,3,3,3,3,3
Brockton / Parkdale Village / Exhibition Place,8,8,8,8,8,8
Business reply mail Processing CentrE,16,16,16,16,16,16


#### Lets find out how many unique categories can be curated from all the returned venues

In [138]:
print('There are {} unique categories'.format(len(toronto_venues['Venue Category'].unique())))

There are 90 unique categories


In [144]:
print(fixed_column)

['Neighborhood', 'Athletics & Sports', 'BBQ Joint', 'Bakery', 'Bank', 'Baseball Field', 'Beer Store', 'Breakfast Spot', 'Brewery', 'Burger Joint', 'Burrito Place', 'Bus Line', 'Bus Station', 'Bus Stop', 'Butcher', 'Café', 'Camera Store', 'Caribbean Restaurant', 'Chinese Restaurant', 'Clothing Store', 'Coffee Shop', 'Construction & Landscaping', 'Convenience Store', 'Deli / Bodega', 'Diner', 'Discount Store', 'Dog Run', 'Dry Cleaner', 'Electronics Store', 'Fast Food Restaurant', 'Fish & Chips Shop', 'Food & Drink Shop', 'Food Truck', 'Fried Chicken Joint', 'Furniture / Home Store', 'Gas Station', 'Gastropub', 'Golf Course', 'Grocery Store', 'Gym', 'Gym / Fitness Center', 'Hardware Store', 'Historic Site', 'History Museum', 'Home Service', 'Ice Cream Shop', 'Indian Restaurant', 'Intersection', 'Italian Restaurant', 'Japanese Restaurant', 'Latin American Restaurant', 'Liquor Store', 'Metro Station', 'Mobile Phone Shop', 'Moving Target', 'Music School', 'Nightclub', 'Other Nightlife', 'Out

## Analyse Each Neighborhood

In [140]:
#one hot encoding
toronto_onehot=pd.get_dummies(toronto_venues[['Venue Category']],prefix="",prefix_sep='')

#add neighborhood column back to dataframe
toronto_onehot['Neighborhood']=toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_column=[toronto_onehot.columns[-1]]+list(toronto_onehot.columns[:-1])
toronto_onehot=toronto_onehot[fixed_column]
toronto_onehot.head()

,Neighborhood,Athletics & Sports,BBQ Joint,Bakery,Bank,Baseball Field,Beer Store,Breakfast Spot,Brewery,Burger Joint,...,Sports Club,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theme Park,Vietnamese Restaurant,Wings Joint,Yoga Studio,Zoo
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [145]:
toronto_onehot.shape

(710, 91)

### Lets group rows by neighborhood and taking the mean of the frequency of each category

In [146]:
toronto_grouped=toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Athletics & Sports,BBQ Joint,Bakery,Bank,Baseball Field,Beer Store,Breakfast Spot,Brewery,Burger Joint,...,Sports Club,Supermarket,Supplement Shop,Tea Room,Thai Restaurant,Theme Park,Vietnamese Restaurant,Wings Joint,Yoga Studio,Zoo
0,Agincourt,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
1,Alderwood / Long Branch,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.250000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000,0.000000
2,Bathurst Manor / Wilson Heights / Downsview North,0.000000,0.000000,0.000000,0.000000,0.0,0.052632,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.052632,0.000000,0.000,0.000000
3,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
4,Bedford Park / Lawrence Manor East,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
5,Berczy Park,0.000000,0.000000,0.050000,0.000000,0.0,0.050000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000,0.000000
6,Birch Cliff / Cliffside West,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
7,Brockton / Parkdale Village / Exhibition Place,0.000000,0.000000,0.000000,0.125000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.125,0.000000
8,Business reply mail Processing CentrE,0.000000,0.000000,0.062500,0.062500,0.0,0.000000,0.000000,0.000000,0.000000,...,0.0,0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000
9,CN Tower / King and Spadina / Railway Lands / ...,0.000000,0.000000,0.076923,0.000000,0.0,0.000000,0.000000,0.000000,0.076923,...,0.0,0.000000,0.076923,0.000000,0.000000,0.000000,0.000000,0.076923,0.000,0.000000


### lets print each neighborhood with top 5 most common venues

In [149]:
num_top_venues=5
for hood in toronto_grouped['Neighborhood']:
    print("-----"+hood+"-----")
    temp=toronto_grouped[toronto_grouped['Neighborhood']==hood].T.reset_index()
    temp.columns=['venue','freq']
    temp=temp.iloc[1:]
    temp['freq']=temp['freq'].astype(float)
    temp=temp.round({'freq':2})
    print(temp.sort_values('freq',ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

-----Agincourt-----
                venue  freq
0       Shopping Mall  0.25
1        Skating Rink  0.25
2     Other Nightlife  0.25
3                Café  0.25
4  Athletics & Sports  0.00


-----Alderwood / Long Branch-----
                   venue  freq
0  Vietnamese Restaurant  0.25
1         Breakfast Spot  0.25
2            Coffee Shop  0.25
3           Skating Rink  0.25
4     Athletics & Sports  0.00


-----Bathurst Manor / Wilson Heights / Downsview North-----
                  venue  freq
0         Grocery Store  0.11
1         Shopping Mall  0.11
2           Coffee Shop  0.11
3           Pizza Place  0.11
4  Gym / Fitness Center  0.05


-----Bayview Village-----
                   venue  freq
0            Bus Station  0.33
1          Metro Station  0.33
2           Skating Rink  0.33
3  Outdoors & Recreation  0.00
4                    Pub  0.00


-----Bedford Park / Lawrence Manor East-----
                venue  freq
0                Pool   1.0
1  Athletics & Sports   0.0
2  

                 venue  freq
0         Intersection  0.25
1       Discount Store  0.25
2          Golf Course  0.25
3               Bakery  0.25
4  Japanese Restaurant  0.00


-----Humewood-Cedarvale-----
            venue  freq
0     Gas Station  0.33
1     Pizza Place  0.33
2            Park  0.33
3  Rental Service  0.00
4             Pub  0.00


-----India Bazaar / The Beaches West-----
                 venue  freq
0             Bus Stop  0.25
1                 Park  0.25
2               Bakery  0.25
3    Convenience Store  0.25
4  Rental Car Location  0.00


-----Islington Avenue-----
               venue  freq
0               Park  0.33
1           Bus Stop  0.17
2             Bakery  0.17
3  Convenience Store  0.17
4       Skating Rink  0.17


-----Kennedy Park / Ionview / East Birchmount Park-----
                 venue  freq
0         Intersection  0.25
1       Discount Store  0.25
2          Golf Course  0.25
3               Bakery  0.25
4  Japanese Restaurant  0.00


-----Ken

4              Bus Stop  0.05


-----Studio District-----
            venue  freq
0             Spa  0.08
1        Pharmacy  0.08
2    Liquor Store  0.08
3            Bank  0.08
4  Ice Cream Shop  0.08


-----Summerhill West / Rathnelly / South Hill / Forest Hill SE / Deer Park-----
               venue  freq
0             Bakery  0.12
1               Bank  0.12
2  Convenience Store  0.12
3     Hardware Store  0.12
4              Plaza  0.12


-----The Annex / North Midtown / Yorkville-----
                  venue  freq
0     Indian Restaurant  0.17
1    Salon / Barbershop  0.17
2                  Park  0.17
3  Fast Food Restaurant  0.17
4               Dog Run  0.17


-----The Beaches-----
               venue  freq
0  Indian Restaurant  0.25
1           Bus Line  0.25
2         Playground  0.25
3            Dog Run  0.25
4       Liquor Store  0.00


-----The Danforth West / Riverdale-----
                   venue  freq
0             Playground  0.50
1         Hardware Store  0.25
2  

## Lets put that into a Pandas DataFrame

First lets write a function to sort venues in descending order

In [166]:
def return_most_common_venues(row,num_top_venues):
    row_categories=row.iloc[1:]
    row_categories_sorted=row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [167]:
x=np.arange(num_top_venues)
x

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [168]:
num_top_venues=10
indicators=['st','nd','rd']
columns=['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} most common venue'.format(ind+1,indicators[ind]))
    except:
        columns.append('{}th most common venue'.format(ind+1))
neighborhood_venue_sorted=pd.DataFrame(columns=columns)
neighborhood_venue_sorted['Neighborhood']=toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhood_venue_sorted.iloc[ind,1:]=return_most_common_venues(toronto_grouped.iloc[ind,:],num_top_venues)
neighborhood_venue_sorted.head()

,Neighborhood,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,Agincourt,Shopping Mall,Café,Skating Rink,Other Nightlife,Zoo,Deli / Bodega,Discount Store,Dog Run,Dry Cleaner,Electronics Store
1,Alderwood / Long Branch,Vietnamese Restaurant,Coffee Shop,Breakfast Spot,Skating Rink,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store
2,Bathurst Manor / Wilson Heights / Downsview North,Coffee Shop,Pizza Place,Grocery Store,Shopping Mall,Gas Station,Fast Food Restaurant,Road,Caribbean Restaurant,Fried Chicken Joint,Discount Store
3,Bayview Village,Bus Station,Metro Station,Skating Rink,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
4,Bedford Park / Lawrence Manor East,Pool,Zoo,Deli / Bodega,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop


## Clustering

#### Running kmeans to cluster neighborhood into 5 clusters

In [169]:
cluster=5
toronto_group_cluster=toronto_grouped.drop('Neighborhood',1)
model=KMeans(n_clusters=cluster,random_state=5).fit(toronto_group_cluster)
model.labels_

array([2, 2, 2, 3, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 1, 2, 3, 0, 2, 1, 0, 1,
       2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 1, 2, 2, 2, 2, 2, 2, 1, 1, 1, 2,
       4, 2, 0, 2, 1, 2, 0, 4, 2, 2, 2, 0, 0, 1, 2, 2, 2, 1, 2, 2, 4, 4,
       2, 1, 3, 4, 2, 1, 2, 4, 1, 2, 2, 2, 2, 2, 2, 2, 4, 1, 2, 2, 2, 2,
       4, 2, 1, 0, 2, 2, 2, 2, 2, 2])

#### Lets create a new data Frame that includes cluster as well as the top 10 venues of each neighborhood 

In [170]:
#add cluster labels
neighborhood_venue_sorted.insert(0,'Cluster_Label',model.labels_)
toronto_merged=toronto_df
toronto_merged=toronto_merged.join(neighborhood_venue_sorted.set_index('Neighborhood'),on='Neighborhood')
toronto_merged.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2,Coffee Shop,Vietnamese Restaurant,Gym / Fitness Center,Pizza Place,Grocery Store,Beer Store,Road,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,2,Intersection,Bakery,Discount Store,Golf Course,Zoo,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
2,M5A,Downtown Toronto,Regent Park / Harbourfront,43.654260,-79.360636,4,Italian Restaurant,History Museum,Dog Run,Fried Chicken Joint,Discount Store,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop
3,M6A,North York,Lawrence Manor / Lawrence Heights,43.718518,-79.464763,0,Construction & Landscaping,Coffee Shop,Furniture / Home Store,Outdoors & Recreation,Zoo,Food Truck,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,43.662301,-79.389494,2,Ice Cream Shop,Spa,Convenience Store,Coffee Shop,Restaurant,Camera Store,Shopping Mall,Bus Stop,Pharmacy,Liquor Store


#### Dropping the rows that have no data

In [171]:
toronto_merged['Cluster_Label']=toronto_merged.Cluster_Label.astype(int)

In [175]:
map_cluster=folium.Map(location=[latitude_tor,longitude_tor],zoom_start=11)
# set colors scheme for clusters
x=np.arange(cluster)
ys=[i+x+(i*x)**2 for i in range(cluster)]
color_array=cm.rainbow(np.linspace(0,1,len(ys)))
rainbow=[colors.rgb2hex(i) for i in color_array]

marker_colors=[]
for lat,lon,poi,cluster in zip(toronto_merged['Latitude'],toronto_merged['Longitude'],toronto_merged['Neighborhood'],toronto_merged['Cluster_Label']):
    label=folium.Popup(str(poi)+' Cluster '+str(cluster),parse_html=True)
    folium.CircleMarker(
    [lat,lon],
    radius=5,
    popup=label,
    color=rainbow[cluster-1],
    fill=True,
    fill_color=rainbow[cluster-1],
    fill_opacity=.7).add_to(map_cluster)
map_cluster



## Examining Clusters

##### CLuster 1

In [176]:
toronto_merged.loc[toronto_merged['Cluster_Label']==0,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
3,North York,0,Construction & Landscaping,Coffee Shop,Furniture / Home Store,Outdoors & Recreation,Zoo,Food Truck,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
6,Scarborough,0,Furniture / Home Store,Print Shop,Gas Station,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
49,North York,0,Furniture / Home Store,Skating Rink,Outdoors & Recreation,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
51,Scarborough,0,Furniture / Home Store,Construction & Landscaping,Outdoors & Recreation,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
67,Central Toronto,0,Park,Skating Rink,Furniture / Home Store,Outdoors & Recreation,Zoo,Food & Drink Shop,Discount Store,Dog Run,Dry Cleaner,Electronics Store
71,Scarborough,0,Furniture / Home Store,Plaza,Tea Room,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
73,Central Toronto,0,Furniture / Home Store,Construction & Landscaping,Outdoors & Recreation,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant


In [177]:
toronto_merged.loc[toronto_merged['Cluster_Label']==1,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
5,Etobicoke,1,Park,Convenience Store,Bus Stop,Bakery,Skating Rink,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
8,East York,1,Breakfast Spot,Park,Zoo,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
10,North York,1,Park,Moving Target,Sports Club,Snack Place,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
16,York,1,Pizza Place,Park,Gas Station,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
23,East York,1,Park,Snack Place,Moving Target,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
25,Downtown Toronto,1,Dog Run,Fast Food Restaurant,Park,Bus Stop,Zoo,Food Truck,Discount Store,Dry Cleaner,Electronics Store,Fish & Chips Shop
29,East York,1,Pharmacy,Breakfast Spot,Park,Fish & Chips Shop,Zoo,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
31,West Toronto,1,Convenience Store,Bakery,Park,Bus Stop,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
32,Scarborough,1,Park,Food & Drink Shop,Zoo,Deli / Bodega,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
44,Scarborough,1,Park,Outdoors & Recreation,Zoo,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop


In [178]:
toronto_merged.loc[toronto_merged['Cluster_Label']==2,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
0,North York,2,Coffee Shop,Vietnamese Restaurant,Gym / Fitness Center,Pizza Place,Grocery Store,Beer Store,Road,Sandwich Place,Fast Food Restaurant,Fish & Chips Shop
1,North York,2,Intersection,Bakery,Discount Store,Golf Course,Zoo,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
4,Downtown Toronto,2,Ice Cream Shop,Spa,Convenience Store,Coffee Shop,Restaurant,Camera Store,Shopping Mall,Bus Stop,Pharmacy,Liquor Store
7,North York,2,Home Service,Tea Room,Park,Food & Drink Shop,Golf Course,Discount Store,Intersection,Bakery,Gym / Fitness Center,Dog Run
9,Downtown Toronto,2,Bank,Supermarket,Camera Store,Spa,Zoo,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store
13,North York,2,Home Service,Tea Room,Park,Food & Drink Shop,Golf Course,Discount Store,Intersection,Bakery,Gym / Fitness Center,Dog Run
14,East York,2,Pizza Place,Grocery Store,Gas Station,Skate Park,Skating Rink,Soccer Field,Zoo,Fish & Chips Shop,Diner,Discount Store
17,Etobicoke,2,Coffee Shop,Pub,Fast Food Restaurant,Japanese Restaurant,Restaurant,Caribbean Restaurant,Sandwich Place,Bus Stop,Grocery Store,Thai Restaurant
18,Scarborough,2,Athletics & Sports,Liquor Store,Pharmacy,Gas Station,Food Truck,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
19,East Toronto,2,Bus Line,Indian Restaurant,Dog Run,Playground,Furniture / Home Store,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop


In [179]:
toronto_merged.loc[toronto_merged['Cluster_Label']==3,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
12,Scarborough,3,Hardware Store,Metro Station,Bus Station,Zoo,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant
39,North York,3,Bus Station,Metro Station,Skating Rink,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
82,Scarborough,3,Hardware Store,Metro Station,Bus Station,Zoo,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant


In [180]:
toronto_merged.loc[toronto_merged['Cluster_Label']==4,toronto_merged.columns[[1]+list(range(5,toronto_merged.shape[1]))]]

,Borough,Cluster_Label,1st most common venue,2nd most common venue,3rd most common venue,4th most common venue,5th most common venue,6th most common venue,7th most common venue,8th most common venue,9th most common venue,10th most common venue
2,Downtown Toronto,4,Italian Restaurant,History Museum,Dog Run,Fried Chicken Joint,Discount Store,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop
11,Etobicoke,4,History Museum,Coffee Shop,Italian Restaurant,Pizza Place,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
15,Downtown Toronto,4,History Museum,Italian Restaurant,Clothing Store,Pizza Place,Park,Food & Drink Shop,Discount Store,Dog Run,Dry Cleaner,Electronics Store
30,Downtown Toronto,4,History Museum,Coffee Shop,Italian Restaurant,Pizza Place,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
81,West Toronto,4,History Museum,Italian Restaurant,Clothing Store,Park,Fried Chicken Joint,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
84,Downtown Toronto,4,Italian Restaurant,History Museum,Dog Run,Fried Chicken Joint,Discount Store,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop
85,Scarborough,4,Italian Restaurant,Butcher,Zoo,Fried Chicken Joint,Discount Store,Dog Run,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop
98,Etobicoke,4,Italian Restaurant,History Museum,Dog Run,Fried Chicken Joint,Discount Store,Dry Cleaner,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop
